In [124]:
from google.colab import drive
drive.mount('/content/gdrive')
dataset_path = "gdrive/My Drive/Colab Notebooks/DEEP LEARNING/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [125]:
from scipy.io import loadmat
import pandas as pd
from matplotlib import pyplot
from pandas import Series
import numpy as np
mat = loadmat(dataset_path + 'Xtrain.mat')
mdata = mat['Xtrain']
series = pd.DataFrame(mdata, columns={"t"})
series

,t
0,86
1,141
2,95
3,41
4,22
...,...
995,61
996,20
997,12
998,13


In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
dataset = series.values
dataset = series.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#train, tmp = train_test_split(dataset, test_size=0.2, random_state=23)
#test, val = train_test_split(tmp, test_size = 0.5, random_state = 23)

In [142]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
 
look_back = 12
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
#valX, valY = create_dataset(val, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
#valX = np.reshape(valX, (valX.shape[0], 1, valX.shape[1]))

800 200


In [143]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
def LSTM_model():
  model = Sequential()
  model.add(LSTM(20, input_shape=(1, look_back)))
  model.add(Dropout(0.2))
  model.add(Dense(1))
  
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

model = LSTM_model()
model.fit(trainX, trainY, epochs=1000, batch_size=20, verbose=2) #need to add validation set


Epoch 1/1000
 - 0s - loss: 0.0568
Epoch 2/1000
 - 0s - loss: 0.0348
Epoch 3/1000
 - 0s - loss: 0.0230
Epoch 4/1000
 - 0s - loss: 0.0173
Epoch 5/1000
 - 0s - loss: 0.0151
Epoch 6/1000
 - 0s - loss: 0.0135
Epoch 7/1000
 - 0s - loss: 0.0128
Epoch 8/1000
 - 0s - loss: 0.0121
Epoch 9/1000
 - 0s - loss: 0.0105
Epoch 10/1000
 - 0s - loss: 0.0096
Epoch 11/1000
 - 0s - loss: 0.0097
Epoch 12/1000
 - 0s - loss: 0.0098
Epoch 13/1000
 - 0s - loss: 0.0092
Epoch 14/1000
 - 0s - loss: 0.0087
Epoch 15/1000
 - 0s - loss: 0.0091
Epoch 16/1000
 - 0s - loss: 0.0079
Epoch 17/1000
 - 0s - loss: 0.0076
Epoch 18/1000
 - 0s - loss: 0.0077
Epoch 19/1000
 - 0s - loss: 0.0076
Epoch 20/1000
 - 0s - loss: 0.0082
Epoch 21/1000
 - 0s - loss: 0.0080
Epoch 22/1000
 - 0s - loss: 0.0076
Epoch 23/1000
 - 0s - loss: 0.0077
Epoch 24/1000
 - 0s - loss: 0.0069
Epoch 25/1000
 - 0s - loss: 0.0074
Epoch 26/1000
 - 0s - loss: 0.0066
Epoch 27/1000
 - 0s - loss: 0.0067
Epoch 28/1000
 - 0s - loss: 0.0067
Epoch 29/1000
 - 0s - loss: 0

In [145]:
import math
from sklearn.metrics import mean_squared_error

trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
valX, valY = create_dataset(val, look_back)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
valX = np.reshape(valX, (valX.shape[0], 1, valX.shape[1]))
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY= trainY.reshape(-1, 1)
trainY = scaler.inverse_transform(trainY)
testPredict = scaler.inverse_transform(testPredict)
testY= testY.reshape(-1, 1)
testY = scaler.inverse_transform(testY)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 3.09 RMSE
Test Score: 2.41 RMSE


In [184]:
#predicting next 200 values
mat = loadmat(dataset_path + 'Xtrain.mat')
mdata = mat['Xtrain']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(mdata)
data_scale = scaler.transform(mdata)
series = pd.DataFrame(data_scale[-look_back:].reshape(1, look_back))
series = series.values
series = np.reshape(series, (series.shape[0], 1, series.shape[1]))


prediction = []
for i in range(200):
  y_pred = model.predict(series)
  y_pred = y_pred[0]
  prediction.append(y_pred)
  series=np.delete(series, 0, 2)
  #series = next_element.shift(-1, axis = 1)
  series= np.append(series, y_pred)
  series = np.reshape(series, (1, 1, series.shape[0]))
  print (y_pred)
prediction = scaler.inverse_transform(prediction)
prediction

[0.27905494]
[0.69233114]
[0.46322203]
[0.1323975]
[0.05215742]
[0.04434404]
[0.07173806]
[0.19686289]
[0.5387903]
[0.56753147]
[0.21117382]
[0.07558183]
[0.04597253]
[0.05673547]
[0.13485502]
[0.40257174]
[0.6342075]
[0.31561145]
[0.10810257]
[0.05525289]
[0.05285472]
[0.10060775]
[0.28783512]
[0.61067855]
[0.4317081]
[0.15009312]
[0.06561476]
[0.05215897]
[0.08186656]
[0.21360677]
[0.5290126]
[0.51983786]
[0.20535421]
[0.08246798]
[0.05548176]
[0.07063218]
[0.16292633]
[0.4322564]
[0.57280695]
[0.27601352]
[0.10586008]
[0.06150585]
[0.06480366]
[0.12982729]
[0.34546763]
[0.58221877]
[0.35327694]
[0.13412315]
[0.06873888]
[0.06205696]
[0.10768549]
[0.2767418]
[0.55486107]
[0.4249243]
[0.16747089]
[0.07826904]
[0.06217979]
[0.09289686]
[0.2239889]
[0.502594]
[0.48288155]
[0.20789483]
[0.09142881]
[0.06485121]
[0.08348466]
[0.1848835]
[0.4400284]
[0.52102125]
[0.25521648]
[0.10828663]
[0.06920186]
[0.0776386]
[0.15619433]
[0.3792756]
[0.5377939]
[0.30602276]
[0.1279245]
[0.07449446]
[0.

array([[ 72.60089976],
       [177.15977722],
       [119.19517279],
       [ 35.49656817],
       [ 15.19582649],
       [ 13.21904156],
       [ 20.14973027],
       [ 51.8063115 ],
       [138.31394225],
       [145.58546102],
       [ 55.42697583],
       [ 21.12220205],
       [ 13.6310491 ],
       [ 16.35407414],
       [ 36.11831932],
       [103.85064965],
       [162.45449424],
       [ 81.84969732],
       [ 29.34994958],
       [ 15.97898237],
       [ 15.37224334],
       [ 27.45376146],
       [ 74.82228565],
       [156.50167406],
       [111.22214866],
       [ 39.9735602 ],
       [ 18.60053426],
       [ 15.19621857],
       [ 22.71224028],
       [ 56.04251403],
       [135.84019297],
       [133.51897764],
       [ 53.95461607],
       [ 22.86439911],
       [ 16.03688571],
       [ 19.86994206],
       [ 43.22036178],
       [111.36086935],
       [146.92015946],
       [ 71.83142141],
       [ 28.78259944],
       [ 17.56098109],
       [ 18.39532596],
       [ 34

In [169]:
series = pd.DataFrame(data_scale[-look_back:].reshape(1, look_back))
series = series.values
series = np.reshape(series, (series.shape[0], 1, series.shape[1]))
series.shape
series=np.delete(series, 0, 2)
series.shape
series= np.append(series, 0)
series.shape
series = np.reshape(series, (1, 1, series.shape[0]))
series.shape

(1, 1, 12)